In [62]:
import pandas as pd
import numpy as np
import re
from prettytable import PrettyTable


from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

df_train = pd.read_csv(r'C:\Users\higaj\Desktop\py\used_cars\train.csv')
df_test = pd.read_csv(r'C:\Users\higaj\Desktop\py\used_cars\test.csv')

table = PrettyTable()
table.field_names = ['Variable_name', 'nUnique', 'DTypes']


for i in df_train.columns:
    var = i 
    nunique = df_train[i].nunique()
    dtyp = df_train[i].dtypes
    table.add_row([var, nunique, dtyp])


table.add_column('Work Status',['DROPPED', 'later', 'later', 'NUMERICAL', 'NUMERICAL', 'CATEGORICAL', 'later', 'later', 'later', 'later','NUMERICAL', 'NUMERICAL', 'Y'])
print(table)
## Based on the information from table below, classify:
# id - Drop column
# y = Price
# Numeric variables: Model_year, milage, accident(make boolean), clean_title(make boolean)
# Categorical variables: fuel_type


# Model + brand: segment based on average price maybe??
# transmission
# Engine: super complex, separate into various columns

# ignore for now: int_col / ext_col

+---------------+---------+--------+-------------+
| Variable_name | nUnique | DTypes | Work Status |
+---------------+---------+--------+-------------+
|       id      |  188533 | int64  |   DROPPED   |
|     brand     |    57   | object |    later    |
|     model     |   1897  | object |    later    |
|   model_year  |    34   | int64  |  NUMERICAL  |
|     milage    |   6651  | int64  |  NUMERICAL  |
|   fuel_type   |    7    | object | CATEGORICAL |
|     engine    |   1117  | object |    later    |
|  transmission |    52   | object |    later    |
|    ext_col    |   319   | object |    later    |
|    int_col    |   156   | object |    later    |
|    accident   |    2    | object |  NUMERICAL  |
|  clean_title  |    1    | object |  NUMERICAL  |
|     price     |   1569  | int64  |      Y      |
+---------------+---------+--------+-------------+


In [63]:
### Drop ID column
df_train.drop(columns='id', inplace=True)
df_test.drop(columns='id', inplace=True)


### Clean the 'Accident' column
df_train['accident'].unique()
df_train.loc[df_train['accident'] == 'None reported', 'accidentbool'] = False
df_train.loc[df_train['accident'] == 'At least 1 accident or damage reported', 'accidentbool'] = True
df_train.drop(columns='accident', inplace = True)

df_test.loc[df_test['accident'] == 'None reported', 'accidentbool'] = False
df_test.loc[df_test['accident'] == 'At least 1 accident or damage reported', 'accidentbool'] = True
df_test.drop(columns='accident', inplace = True)


### Clean the 'clean_title' column
df_train['clean_title'].unique()
df_train.loc[df_train['clean_title'] == 'Yes', 'cleanbool'] = True
df_train.drop(columns='clean_title', inplace = True)

df_test.loc[df_test['clean_title'] == 'Yes', 'cleanbool'] = True
df_test.drop(columns='clean_title', inplace = True)



############ Clean the 'Engine' column
def hp_extract(row):
    if 'HP' in row:
        hpower, rest = row.split('HP', 1)
        return pd.Series([hpower.strip(), rest.strip()])
    else:
        return pd.Series([np.nan, row])

def litters_extract(row):
    pattern = r'(\d+(?:\.\d+)?L)'  # Regex to capture only the liter value (integer or decimal)
    match = re.search(pattern, row)  # Search for the pattern in the row
    
    if match:  # If a match is found
        litrage = match.group(1).strip()  # Extract the matched liter value (e.g., '4.75L' or '2L')
        rest = row.replace(litrage, '').strip()  # Remove the matched liter value from the row
        return pd.Series([litrage, rest])  # Return the extracted liter and remaining text
    else:
        return pd.Series([np.nan, row])  # If no match, return NaN for litrage

def litters_extract2(row):
    if 'Liter' in row:
        hpower, rest = row.split('Liter', 1)

        return pd.Series([hpower.strip(), rest.strip()])
    else:
        return pd.Series([np.nan, row])
    
def get_fuel(fuel, row):
    if fuel in row:
        eng = row.replace(fuel, '').strip()
        return eng
    else:
        return row  
                      
    
# Keep this in order! Bc working in order stripping the texts
df_train[['horse_power', 'engine']] = df_train['engine'].apply(hp_extract)
df_train[['litrage', 'engine']] = df_train['engine'].apply(litters_extract)
df_train[['litrage2', 'engine']] = df_train['engine'].apply(litters_extract2)

df_train['litrage'] = df_train['litrage'].astype(str).str.rstrip('L')
df_train['litrage'] = df_train['litrage'].astype(float)
df_train['litrage2'] = df_train['litrage2'].astype(float)

df_train['litrage'] = df_train['litrage'].fillna(df_train['litrage2'])
df_train.drop(columns='litrage2', inplace = True)
#############################################################################

# Now, stripping the fuel type. There is already a 'fuel' column!

fueltp_list = ['Gasoline Fuel', 'Gasoline/Mild Electric Hybrid', 'Diesel Fuel', 'Gas/Electric Hybrid', 'Flex Fuel Capability',
              'Flexible Fuel', 'Diesel', 'Plug-In Electric/Gas', 'Hybrid']

for fuel in fueltp_list:
    df_train['engine'] = df_train['engine'].apply(lambda row: get_fuel(fuel, row))

df_train

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horse_power,litrage
0,0,MINI,Cooper S Base,2007,213000,Gasoline,4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6
1,1,Lincoln,LS V8,2002,143250,Gasoline,8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,8 Cylinder Engine Flex Fuel Capability,A/T,Blue,Gray,None reported,Yes,13900,320.0,5.3
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,5.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500,420.0,6.2
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000,385.0,3.0
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900,469.0,4.0
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900,NaN,3.0


In [64]:
df_train

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horse_power,litrage
0,0,MINI,Cooper S Base,2007,213000,Gasoline,4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6
1,1,Lincoln,LS V8,2002,143250,Gasoline,8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,8 Cylinder Engine Flex Fuel Capability,A/T,Blue,Gray,None reported,Yes,13900,320.0,5.3
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,5.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500,420.0,6.2
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000,385.0,3.0
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900,469.0,4.0
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900,NaN,3.0


In [66]:
df_train

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,horse_power,litrage
0,0,MINI,Cooper S Base,2007,213000,Gasoline,4 Cylinder Engine,A/T,Yellow,Gray,None reported,Yes,4200,172.0,1.6
1,1,Lincoln,LS V8,2002,143250,Gasoline,8 Cylinder Engine,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,252.0,3.9
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,8 Cylinder Engine,A/T,Blue,Gray,None reported,Yes,13900,320.0,5.3
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,8 Cylinder Engine,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,420.0,5.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,4 Cylinder Engine,7-Speed A/T,Black,Beige,None reported,Yes,97500,208.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,8 Cylinder Engine,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500,420.0,6.2
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,V6 Cylinder Engine,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000,385.0,3.0
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,8 Cylinder Engine,7-Speed A/T,White,Black,None reported,Yes,86900,469.0,4.0
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900,NaN,3.0


In [68]:
df_train.to_csv(r'C:\Users\higaj\Desktop\auxiliarrr.csv')

C:\Users\higaj\AppData\Local\Temp\ipykernel_10360\3880557373.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train['accident'] == 'None reported', 'accidentbool'] = False
C:\Users\higaj\AppData\Local\Temp\ipykernel_10360\3880557373.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_test.loc[df_test['accident'] == 'None reported', 'accidentbool'] = False


C:\Users\higaj\AppData\Local\Temp\ipykernel_10360\774671184.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train['clean_title'] == 'Yes', 'cleanbool'] = True
C:\Users\higaj\AppData\Local\Temp\ipykernel_10360\774671184.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_test.loc[df_test['clean_title'] == 'Yes', 'cleanbool'] = True


In [ ]:
# Define the Tables
X = df_train
X = X.drop(columns='price')
y = df_train.price

# Define the columns that are going to be used
numerical_cols = ['model_year', 'milage', 'brand_avg', 'model_avg', 'engine_avg', 'transmission_avg']
categorical_cols = ['fuel_type', 'cleanbool', 'accidentbool']

# Separate test and validation of the df_train dataframe
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Defining strategies for numerical and categorical columns
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle the preprocessor

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Modeling
model = RandomForestRegressor(n_estimators = 100, random_state=0)
clf = Pipeline(steps=[('preprocessor', preprocessor),
                  ('model', model)
                 ])

clf.fit(X_train, y_train)
preds = clf.predict(X_valid)
print(np.sqrt(mean_squared_error(preds, y_valid)))

In [105]:
## Run model and make it spit the submissioN!
X = df_test
prediccion = clf.predict(X)

submission = pd.DataFrame()
temp = pd.read_csv(r'C:\Users\higaj\Desktop\py\used_cars\sample_submission.csv')
submission['id'] = temp['id']
submission['price'] = prediccion
submission.to_csv('submission.csv',index=False)